# 1. 练手项目

In [1]:
import cv2
import os
import random
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, Input
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.optimizers import SGD
from keras.applications.vgg16 import preprocess_input
from keras.optimizers import RMSprop, Adam
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.utils import to_categorical

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit

%matplotlib inline

Using TensorFlow backend.


##  1.1加载数据

In [2]:
#读取图片
def read_and_process_image(data_dir, width = 64, height = 64, channels = 3, preprocess = False):
    train_images = [data_dir + i for i in os.listdir(data_dir) if i != '.DS_Store']

    random.shuffle(train_images)
    
    def read_image(file_path, preprocess):
        img = image.load_img(file_path)
        target = (height, width)
        img = img.resize(target)
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        if preprocess:
            x = preprocess_input(x)
        return x

    def prep_data(images, preprocess):
        count = len(images)
        data = np.ndarray((count, width, height, channels), dtype=np.float32)

        for i, image_file in enumerate(images):
            image = read_image(image_file, preprocess)
            data[i] = image
    
        return data
    
    def read_labels(file_path):
        #Using 1 to represent dog and 0 for cat
        labels = []
        label_encoder = LabelEncoder()
        for i in file_path:
            label = i.split('/')[1].split('.')[0].split('_')[0]
            labels.append(label)
        labels = label_encoder.fit_transform(labels)
        
        return labels, label_encoder

    X = prep_data(train_images, preprocess)
    labels, label_encoder = read_labels(train_images)
    
    assert X.shape[0] == len(labels)

    print("Train shape: {}".format(X.shape))
    
    return X, labels, label_encoder

In [3]:
WIDTH = 48
HEIGHT = 48
CHANNELS = 3
X, y, label_encoder = read_and_process_image('input/', width = WIDTH, height = HEIGHT, channels = CHANNELS)

Train shape: (62, 48, 48, 3)


In [4]:
sss = StratifiedShuffleSplit(n_splits=2, test_size=0.3, random_state=0)

In [5]:
sss.get_n_splits(X, y)

2

In [6]:
for train_index, test_index in sss.split(X, y):
    train_X, train_y = X[train_index], y[train_index]
    test_X, test_y = X[test_index], y[test_index]

In [7]:
test_y

array([0, 1, 0, 1, 2, 2, 0, 2, 2, 0, 3, 3, 2, 3, 2, 2, 3, 0, 2],
      dtype=int64)

In [8]:
#转lable
train_y = to_categorical(train_y)
test_y = to_categorical(test_y)

In [9]:
train_y.shape


(43, 4)

In [10]:
#定义Vgg16模型
def vgg16_model(input_shape = (WIDTH, HEIGHT, CHANNELS)):
    vgg16 = VGG16(include_top = False, weights = 'imagenet', input_shape = input_shape)
    
    for layer in vgg16.layers:
        layer.trainable = False
    
    last = vgg16.output
    
    x = Flatten()(last)
    x = Dense(256, activation = 'relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(4, activation='softmax')(x)
    
    model = Model(inputs = vgg16.input, outputs=x)
    
    return model

In [11]:
#加载VGG16模型
model_vgg16 = vgg16_model()

A local file was found, but it seems to be incomplete or outdated because the auto file hash does not match the original value of 6d6bbae143d832006294945121d1f1fc so we will re-download the data.
58892288/58889256 [==============================] - 107s 2us/step


In [12]:
#查看模型参数
model_vgg16.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 48, 48, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 48, 48, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 48, 48, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 24, 24, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 24, 24, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 24, 24, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 12, 12, 128)       0         
__________

In [13]:
#定义训练损失函数 优化器 精确度
model_vgg16.compile(loss='categorical_crossentropy', optimizer = Adam(0.0001), metrics=['accuracy'])

In [14]:
#喂入数据
history = model_vgg16.fit(train_X, train_y, validation_data=(test_X,test_y), epochs=20, batch_size=64)
#计算错误率
scores = model_vgg16.evaluate(test_X, test_y, verbose=True)
print("VGG-16 Pretrained Model Error: %.2f%%" %(100-scores[1] * 100))

Train on 43 samples, validate on 19 samples
Epoch 1/20
43/43 [==============================] - 7s 157ms/step - loss: 9.8206 - acc: 0.2558 - val_loss: 10.6653 - val_acc: 0.1579
Epoch 2/20
43/43 [==============================] - 5s 122ms/step - loss: 8.4327 - acc: 0.3256 - val_loss: 10.4557 - val_acc: 0.1579
Epoch 3/20
43/43 [==============================] - 5s 119ms/step - loss: 8.3324 - acc: 0.3023 - val_loss: 10.2977 - val_acc: 0.2632
Epoch 4/20
43/43 [==============================] - 6s 129ms/step - loss: 8.9224 - acc: 0.3023 - val_loss: 10.1396 - val_acc: 0.2632
Epoch 5/20
43/43 [==============================] - 5s 119ms/step - loss: 9.8714 - acc: 0.2791 - val_loss: 9.9857 - val_acc: 0.2632
Epoch 6/20
43/43 [==============================] - 5s 126ms/step - loss: 8.6355 - acc: 0.3721 - val_loss: 9.7993 - val_acc: 0.2105
Epoch 7/20
43/43 [==============================] - 5s 117ms/step - loss: 8.8030 - acc: 0.3256 - val_loss: 9.6087 - val_acc: 0.2105
Epoch 8/20
43/43 [==========

In [15]:
#测试准确性
y_test_pred = model_vgg16.predict(test_X)
y_lable = np.argmax(y_test_pred, axis=1)

In [21]:
y_lable[:1]

array([2], dtype=int64)

In [ ]:
for i in range(test_X.shape[0]):
    plt.figure(figsize=(10,5), frameon=True)
    img = test_X[i, :, :, ::-1]
    img = img/255.
    plt.imshow(img)
    plt.title('predict class: {}'.format(label_encoder.inverse_transform([y_label[i]])[0]))
    plt.show()